<a href="https://colab.research.google.com/github/ikanx101/G-Colab/blob/main/Jadwal_interviewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [211]:
#system("sudo apt-get install libglpk-dev")
#install.packages("ROI")
#install.packages("ompr")
#install.packages("ompr.roi")
#install.packages("ROI.plugin.glpk")

In [212]:
library(dplyr)
library(ompr)
library(ompr.roi)
library(ROI.plugin.glpk)
library(ggplot2)

rm(list=ls())

In [213]:
# ada 3 tempat cenloc
# jalan 10 hari
# hari kerja min 3 orang, sabtu minggu jalan min 3 orang
# interviewer hanya boleh kerja max 5 hari berturut2
# ada 16 orang interviewer
# 1 interviewer hanya bisa diassign ke 1 tempat cenloc di tanggal tertentu

# Misal:
# i = 1,2,...,16 banyak interviewer
# c = 1,2,3 banyak tempat
# h = 1,2,...,10 banyak hari
# hk itu dari 1-5, 8-10
# hl itu dari 6-7

# x[i,c,h] = 1 artinya interviewer ke i diassign ke tempat x pada hari h

problem = 
data.frame(
  interviewer = rep(c(1:20),3),
  tempat = c(rep(1,20),rep(2,20),rep(3,20)),
  t1 = 0,
  t2 = 0,
  t3 = 0,
  t4 = 0,
  t5 = 0,
  t6 = 0,
  t7 = 0,
  t8 = 0,
  t9 = 0,
  t10 = 10
)

head(problem)

,interviewer,tempat,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,0,0,0,0,0,0,0,0,0,10
2,2,1,0,0,0,0,0,0,0,0,0,10
3,3,1,0,0,0,0,0,0,0,0,0,10
4,4,1,0,0,0,0,0,0,0,0,0,10
5,5,1,0,0,0,0,0,0,0,0,0,10
6,6,1,0,0,0,0,0,0,0,0,0,10


In [214]:
i_max = 11

model = 
  MIPModel() %>%

  # decision variable
  add_variable(x[i,c,h],
               i = 1:i_max, # orangnya
               c = 1:3,     # tempatnya
               h = 1:10,    # tanggalnya
               type = "binary") %>%
  
  # kalau hari kerja min 1 orang per tempat, per tanggal
  add_constraint(sum_expr(x[i,c,h], i = 1:i_max) >= 1, h = c(1:5,8:10), c = 1:3) %>%

  # kalau hari kerja min 1 orang per tempat, per tanggal
  add_constraint(sum_expr(x[i,c,h], i = 1:i_max) <= 2, h = c(1:5,8:10), c = 1:3) %>%

  # kalau hari libur min 3
  add_constraint(sum_expr(x[i,c,h], i = 1:i_max) >= 3, h = c(6,7), c = 1:3) %>%

  # satu interviewer di satu waktu
  add_constraint(sum_expr(x[i,c,h], c = 1:3) <= 1, i = 1:i_max, h = 1:10) %>%

  # satu orang interviewer gak boleh bekerja lebih dari 5 hari
  add_constraint(sum_expr(x[i,c,h], c = 1:3, h = 1:10) <= 6, i = 1:i_max, c = 1:3) %>%

  # set objective
  set_objective(sum_expr(x[i,c,h],
                  i = 1:i_max,
                  c = 1:3,
                  h = 1:10),
                "max")

model

Mixed integer linear optimization problem
Variables:
  Continuous: 0 
  Integer: 0 
  Binary: 330 
Model sense: maximize 
Constraints: 197 

In [215]:
result <- solve_model(model, with_ROI(solver = "glpk", verbose = TRUE))

<SOLVER MSG>  ----
GLPK Simplex Optimizer, v4.65
197 rows, 330 columns, 1914 non-zeros
      0: obj =  -0.000000000e+00 inf =   4.200e+01 (30)
     87: obj =   4.200000000e+01 inf =   0.000e+00 (0)
*   174: obj =   6.600000000e+01 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Integer Optimizer, v4.65
197 rows, 330 columns, 1914 non-zeros
330 integer variables, all of which are binary
Integer optimization begins...
Long-step dual simplex will be used
+   174: mip =     not found yet <=              +inf        (1; 0)
+   174: >>>>>   6.600000000e+01 <=   6.600000000e+01   0.0% (1; 0)
+   174: mip =   6.600000000e+01 <=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOLUTION FOUND
<!SOLVER MSG> ----


In [216]:
result$objective_value

[1] 66

In [217]:
data = 
  result %>%
  get_solution(x[i,c,h])

In [218]:
library(reshape2)
head(data)

,variable,i,c,h,value
,<chr>,<int>,<int>,<int>,<dbl>
1,x,1,1,1,0
2,x,2,1,1,0
3,x,3,1,1,0
4,x,4,1,1,1
5,x,5,1,1,0
6,x,6,1,1,0


In [219]:
data %>%
  mutate(value = ifelse(value == 1, paste0("tempat-",c),NA)) %>%
  filter(!is.na(value)) %>%
  group_by(c,h) %>%
  summarise(interviewer = paste(i,collapse = ",")) %>%
  ungroup() %>%
  rename(tempat = c,
         tanggal = h) %>%
  dcast(tempat~tanggal,value.var = "interviewer")

`summarise()` regrouping output by 'c' (override with `.groups` argument)



tempat,1,2,3,4,5,6,7,8,9,10
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,"4,8","1,10","1,10","1,11","1,11","1,2,3","4,5,6","4,7","4,8","4,8"
2,"2,9","2,11","2,11",2,2,"4,5,6","7,8,9","5,8","5,9","5,9"
3,"3,5","3,6","3,7","3,7","3,7","7,8,9,10,11","1,10,11","6,9","6,10","6,10"


In [220]:
data %>%
  filter(value == 1) %>%
  group_by(i) %>%
  summarise(bertugas = sum(value)) %>%
  ungroup() %>%
  rename(interviewer = i)

`summarise()` ungrouping output (override with `.groups` argument)



interviewer,bertugas
<int>,<dbl>
1,6
2,6
3,6
4,6
5,6
6,6
7,6
8,6
9,6
